In [ ]:
# Use a pipeline as a high-level helper
import pdb
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration, BitsAndBytesConfig
import torch
from PIL import Image
import time
from llavaAgent import LLaVaAgent


In [ ]:

agent = LLaVaAgent('mistral', '7b')

In [ ]:
import cv2
im = cv2.imread("images_for_vlm/scene1.png")
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
pt = ("You are a robot navigating within an 3-D environment as shown. In the image you see, there are "
                  f"{4} labeled objects. You will be asked to analyze the spatial position of these labeled "
                  f"objects with relation to each other. The red dots on each object are the object's center of mass, "
                  f"which you should use when comparing the position of two objects. From your point of view, "
                  f"answer each question with the"
                  f"descriptors right/left, above/below, in front/behind. If there is no clear spatial difference "
                  f"along a given axis, you can answer 'neutral'")
pt += ("\n1. Where is the led tv in relation to the couch?\n2. Where is the couch in relation to the chandelier?\n3. Where is the chandelier in relation to the kitchen countertop item?")

pt +=  ("\nReason through the task  and describe the 3d layout of the image you see. Output your thinking and at the very end output "
                   "'ANSWER' followed by a json object in the following example format:"
                   "\n{1: ['right', 'above', 'neutral'], 2: ['left', 'neutral', 'in front']}\nThere should be 3 key-value pairs corresponding to the 3 questions")

out, eff = agent.call(im, pt, 3)

In [ ]:
print(out)

In [ ]:
out.index('{')



In [ ]:

d[1][0]

In [ ]:

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf",
                                                          quantization_config=quantization_config,
                                                          device_map="auto",
                                                          torch_dtype=torch.float16, low_cpu_mem_usage=True
                                                          )

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")


In [ ]:
# prepare image and text prompt, using the appropriate prompt template
import cv2
im = cv2.imread("images_for_vlm/scene1.png")
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

image = Image.fromarray(im, mode='RGB')
pt = ("You are a robot navigating within an 3-D environment as shown. In the image you see, there are "
                  f"{4} labeled objects. You will be asked to analyze the spatial position of these labeled "
                  f"objects with relation to each other. The red dots on each object are the object's center of mass, "
                  f"which you should use when comparing the position of two objects. From your point of view, "
                  f"answer each question with the"
                  f"descriptors right/left, above/below, in front/behind. If there is no clear spatial difference "
                  f"along a given axis, you can answer 'neutral'")
pt += ("\n1. Where is the led tv in relation to the couch?\n2. Where is the couch in relation to the chandelier?\n3. Where is the chandelier in relation to the kitchen countertop item?")

pt +=  ("\nReason through the task  and describe the 3d layout of the image you see. Output your thinking and at the very end output "
                   "'ANSWER' followed by a json object in the following example format:"
                   "\n{1: ['right', 'above', 'neutral'], 2: ['left', 'neutral', 'in front']}\nThere should be 3 key-value pairs corresponding to the 3 questions")


prompt = f"[INST] <image>\n{pt} [/INST]"
#prompt = f"A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: <image>\n{pt} ASSISTANT:"

inputs = processor(prompt, image, return_tensors="pt").to(model.device)

print('starting output')
t = time.time()
input_tokens = inputs['input_ids'].shape[1]
print(f'prompt tokens: {input_tokens}')
output = model.generate(**inputs, max_new_tokens=1000)
duration = time.time() - t
print(f'finished output, took {duration} seconds')
output_tokens = output.shape[1]
print(f'output tokens: {output_tokens-input_tokens}')
print(f'efficiency, {(output_tokens-input_tokens)/duration} tokens per second')
print(processor.decode(output[0][input_tokens:], skip_special_tokens=True))

In [ ]:
print(prompt)

In [ ]:

print(type(im))

In [ ]:
im.shape